In [ ]:
import sys
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import log_loss

torch.manual_seed(0)
torch.set_default_dtype(torch.float64)
dtype = torch.float64

def moa_nn_seq(df_x, df_y, df_x2, df_y2, sub_data=None, epochs=1000, lr=1e-3, gpu=False, hidden=1):
    device = torch.device("cpu")
    if gpu:
        device = torch.device("cuda:0")  # Uncomment this to run on GPU
        torch.backends.cuda.matmul.allow_tf32 = False  # Uncomment this to run on GPU
    x = df_x.values
    x = torch.tensor(x, device=device, dtype=dtype)

    y = df_y.values
    y = torch.tensor(y, device=device, dtype=dtype)

    x2 = df_x2.values
    x2 = torch.tensor(x2, device=device, dtype=dtype)
    y2 = None
    if df_y2 is not None:
        y2 = df_y2.values

    if sub_data is not None:
        sub_data = sub_data.values
        sub_data = torch.tensor(sub_data, device=device, dtype=dtype)
    # N, D_in, H, D_out = 64, 1000, 100, 10
    N = x.shape[0]
    D_in = x.shape[1]
    H = hidden
    D_out = y.shape[1]
    print(N, D_in, H, D_out)

#     model = torch.nn.Sequential(
#         torch.nn.Linear(x.shape[1], H),
#         torch.nn.ReLU(),
#         torch.nn.Linear(H, int(H/2)),
#         torch.nn.ReLU(),
#         torch.nn.Linear(int(H/2), y.shape[1])
#     )
    #model.cuda()
    model = torch.nn.Sequential(
      torch.nn.Linear(x.shape[1], H),
      torch.nn.ReLU(),
      torch.nn.Linear(H, y.shape[1]),
    )

    get_loss = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    print('learning_rate', lr)
    for t in range(epochs):
        y_pred = model(x)

        loss = get_loss(y_pred, y)

        if t % (epochs / 10) == ((epochs / 10)-1):
            if y2 is not None:
                model.eval()
                y_pred_test = model(x2)
                y_pred_test = y_pred_test.sigmoid().detach().cpu().numpy()
                logloss2 = log_loss_metric(y2, y_pred_test)
                print(t, loss.item(), logloss2)
                model.train()
            print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    sub_data_pred = None
    if sub_data is not None:
        sub_data_pred = model(sub_data)
        sub_data_pred = sub_data_pred.sigmoid().detach().cpu().numpy()
        return sub_data_pred

feature_data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
scored_data = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

rlist = feature_data[feature_data['cp_type'] == 'ctl_vehicle'].index.tolist()
feature_data = feature_data.drop(index=rlist).reset_index(drop=True)
feature_data = feature_data.drop(['sig_id', 'cp_type'], axis=1)
# feature_data['cp_time'] = feature_data['cp_time'].map({24: 1, 48: 2, 72:3})
feature_data['cp_dose'] = feature_data['cp_dose'].map({'D1': 1, 'D2': 2})
scored_data = scored_data.drop(index=rlist).reset_index(drop=True)
scored_data = scored_data.drop(['sig_id'], axis=1)


test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
rlist = test_features[test_features['cp_type'] == 'ctl_vehicle'].index.tolist()
test_features = test_features.drop(['sig_id', 'cp_type'], axis=1)
test_features['cp_dose'] = test_features['cp_dose'].map({'D1': 1, 'D2': 2})

sub_data_pred = moa_nn_seq(feature_data, scored_data, test_features, None, sub_data=test_features, epochs=1500, lr=1e-2, gpu=False, hidden=400)
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
for x in rlist:
    sub_data_pred[x] = 0
sample_submission.iloc[:,1:] = sub_data_pred
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()




